In [1]:
# in this file we will go through some basic commands used by qutip

# the first thing we need to do is install the qutip package, which you can do with either of the following commands

# pip install qutip
# or
# conda install -c conda-forge qutip      

# we will also need numpy and matplotlib, which you can install with the following commands
# pip install numpy matplotlib  
# or
# conda install numpy matplotlib

In [1]:
# after being installed, we can import these packages
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt

# Define operators on the qubit

In [2]:
# the first thing we are going to do is create a qubit (aka a two-level system TLS)
# to do this, we need to define the orthonormal basis states of the Hilbert space of the qubit.
# we will call these states |e> and |g>, which we can define as follows:

# |e> is the excited state, which we can represent as a column vector [1, 0]
e = qt.basis(2, 0)

# |g> is the ground state, which we can represent as a column vector [0, 1]
g = qt.basis(2, 1)

In [3]:
# lets print out the excited state to see what it looks like

print(e)

Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]]


In [4]:
# it is an instance of the qutip.Qobj class, which is a class that represents a quantum object in qutip.
# TODO: if you are not famililar with classes, instances of a class, methods and attributes, then have a look at some explanations online.

# we can see that 'e' is a column vector with two elements, which corresponds to the two levels of the qubit.
# we can also see that it is normalized, which means that the sum of the squares of the elements is equal to 1.
# this is a requirement for quantum states, as they must be normalized to represent valid quantum states.

# looking at its dimensions (dims) we can see it has 2 rows and 1 column
# notice how the dimensions are stored, though, its not [2, 1] but rather [[2], [1]]
print(e.dims)

[[2], [1]]


In [5]:
# contrast this with the shape of the object, which is stored as a tuple
print(e.shape)

(2, 1)


In [6]:
# we can extract the elements of the vector using the .full() method, which returns a numpy array
print(e.full())

# notice that the elements are complex numbers

[[1.+0.j]
 [0.+0.j]]


In [7]:
# to extract a specific element of the vector, we can use the numpy indexing syntax
# the first index of 0 gets us inside the first set of square brackets
# the second index gets us inside the second set of square brackets, which is where we choose the row we want to extract


print(e.full()[0, 0])  # this will print the first element of the vector, which is 1.0
print(e.full()[1, 0])  # this will print the second element of the vector, which is 0.0

(1+0j)
0j


In [8]:
# we arent going to want to work with these ket vectors directly, since we are interested in representing mixed states, which are matrices and not vectors
# however, the ket vectors are used to define the different elements of the density matrix

# we define these matrix elements as the outer product of the ket vectors

# we will define the matrix elements density matrix of the qubit as follows:
# [ee eg
#  ge gg]
# where 
ee = e * e.dag() # |e><e|
eg = e * g.dag() # |e><g|
ge = g * e.dag() # |g><e|
gg = g * g.dag() # |g><g|


In [9]:
# lets print out each matrix element, and you should see that they are all 2x2 matrices
# and they have a '1' in the position corresponding to the outer product of the ket vectors, and '0's everywhere else
print("ee = ", ee)
print("eg = ", eg)
print("ge = ", ge)
print("gg = ", gg)

# notice the position of the '1' corresponds to where I wrote the outer product in the matrix in the cell above

ee =  Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]]
eg =  Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=False
Qobj data =
[[0. 1.]
 [0. 0.]]
ge =  Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=False
Qobj data =
[[0. 0.]
 [1. 0.]]
gg =  Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 0.]
 [0. 1.]]


In [10]:
# lets check that this has worked. 
# If I wanted to create a matrix that looked like this
# [0.1 0.4+1j
#  0.4-1j 0.9]

# then we would need to do the following 
matrix = 0.1 * ee + (0.4 + 1j) * eg + (0.4 - 1j) * ge + 0.9 * gg

# check
print(matrix)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.1+0.j 0.4+1.j]
 [0.4-1.j 0.9+0.j]]


In [11]:
# since e and g are an orthonormal basis, we can use them to define any operator which acts on the qubit
# for example, we can define the Pauli operators as follows:
# X = e * g.dag() + g * e.dag() # Pauli-X
# Y = 1j * (e * g.dag() - g * e.dag()) # Pauli-Y
# Z = e * e.dag() - g * g.dag() # Pauli-Z   

# we will now use these operators to define the Hamiltonian of the qubit

# Define the qubit Hamiltonian

In [12]:
# TODO: define the Pauli  Z operator
X = e * g.dag() + g * e.dag()  # Pauli X


In [13]:
# TODO: Define the Hamiltonian of the qubit as H = 0.5 * epsilon * Z + 0.5 * delta * X
epsilon = 1.0  # energy difference between the two levels
delta = 0.5    # driving strength between the two levels




# Define operators which act on the Reaction Coordinate 

In [14]:
# notice that the only operators which act on the RC are the raising and lower operators
# before we define these in qutip, we need to choose the dimension of the RC Hilbert space, M,
# which is the same as the number of energy levels we want to include in the RC

M = 5  # number of energy levels in the RC

# we can now define the raising and lowering operators as follows
a = qt.destroy(M)  # lowering operator
adag = a.dag()     # raising operator




# Use the tensor product to define operators on the extended system

In [15]:
# unlike when we write things down on paper (where we sometimes omit the tensor product symbol) in qutip we need to explicitly define the tensor product of the operators

# we can do this using the qt.tensor() function, which takes as arguments the operators we want to tensor product together

# for example, if we want to perform the X operator on the TLS, we need to tell qutip to perform the identity operator on the RC at the same time
# this is because we are treating the extended system as a tensor product of the TLS and the RC

# define the identity operators on the RC and TLS
I_RC = qt.qeye(M)  # identity operator on the RC
I_sys = qt.qeye(2) # identity operator on the TLS

# define an operator which acts non-trivially on the TLS and trivially on the RC
X_ES = qt.tensor(X, I_RC)  # Pauli-X operator on the TLS, nothing on the RC

# define an operator which acts non-trivially on the RC and trivially on the TLS
a_ES = qt.tensor(I_sys, a)  # nothing on the TLS, lowering operator on the RC

# notice that I am being consistent with the order of the operators in the tensor product
# the first operator acts on the TLS, and the second operator acts on the RC

In [16]:
# if you are confused about what is going on, now is a good time to look at the dimensions of the operators
# lets see what the difference between 'X' (which acts on the TLS) and 'X_sys' (which acts on the extended system)
print(X)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 1.]
 [1. 0.]]


In [17]:
# its a 2x2 matrix, which acts only on the TLS

# now lets look at the dimensions of the X_ES operator
print(X_ES)

Quantum object: dims=[[2, 5], [2, 5]], shape=(10, 10), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [18]:
# this is a 2Mx2M matrix (where M =5, which we chose earlier)
# notice how the dimensions are stored, though, its not [2M, 2M] but rather [[2], [M], [2], [M]]
# this can be a bit confusing, but it is just how qutip stores the dimensions of the operators, its basically storing the fact that you are tensoring two operators together

# the dims are dims=[[2, 5], [2, 5]]
# but the shape is shape=(10, 10)



# Define the extended system Hamiltonian

In [19]:
# I will give some random numbers for the values of the energies within the extended system Hamiltonian,

epsilon = 1.0  # energy difference between the two levels
delta = 0.5    # driving strength between the two levels
llambda = 0.1  # coupling strength between the TLS and the RC ('lambda' is a reserved keyword in Python, so we use llambda instead)
Omega = 0.2    # energy of the RC

# can you build the extended system Hamiltonian?
# here is how I might write it down on paper:

$H_{ES} = \frac{\epsilon}{2}\sigma_{z} + \frac{\Delta}{2}\sigma_{x} + \lambda\sigma_{z}\otimes(a+a^{\dagger}) + \Omega a^{\dagger}a$

In [ ]:
# but I have omitted the tensor product symbol, which we need to include in qutip

# TODO: Define the extended system Hamiltonian 
# hint: make sure each object is the same dimension when you try and add them together,
# does it make sense to add a 2x2 matrix to a 2Mx2M matrix? 
# does it make sense to add an MxM matrix to a 2Mx2M matrix?

